In [0]:
!pip install pydrive
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
download = drive.CreateFile({'id': '1-U4nlAp57FOzQL3GoB4CETCbjbluGIKP'})
download.GetContentFile('/content/spatial.7z')


!7z e /content/spatial.7z

!mkdir /content/scenes/
!mkdir /content/scenes/coast/
!mkdir /content/scenes/forest/
!mkdir /content/scenes/highway/
!mkdir /content/nst/
!mkdir /content/nst/styledata/
# !mkdir /content/scenes/insidecity/
#!mkdir /content/scenes/mountain/
#!mkdir /content/scenes/opencountry/
# !mkdir /content/scenes/street/
# !mkdir /content/scenes/tallbuilding/

!mv /content/coast*.jpg /content/scenes/coast
!mv /content/forest*.jpg /content/scenes/forest
!mv /content/highway*.jpg /content/scenes/highway
!mv /content/10power*.jpg /content/nst/styledata
# !mv /content/insidecity*.jpg /content/scenes/insidecity
#!mv /content/mountain*.jpg /content/scenes/mountain
#!mv /content/opencountry*.jpg /content/scenes/opencountry
# !mv /content/street*.jpg /content/scenes/street
# !mv /content/tallbuilding*.jpg /content/scenes/tallbuilding
!rm /content/*.jpg

     |████████████████████████████████| 993kB 5.0MB/s 
  Stored in directory: /root/.cache/pip/wheels/fa/d2/9a/d3b6b506c2da98289e5d417215ce34b696db856643bad779f4
Successfully built pydrive

7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.20GHz (406F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan /content/                   1 file, 127578470 bytes (122 MiB)

Extracting archive: /content/spatial.7z
--
Path = /content/spatial.7z
Type = 7z
Physical Size = 127578470
Headers Size = 28382
Method = LZMA2:24
Solid = +
Blocks = 1

  0%      2% 1        4% 142 - spatial/coast_n295068.jpg                                      5% 143 - spatial/coast_n295080.jpg                                    

In [0]:
# USAGE
# python basic_cnn.py

# import the necessary packages
from keras.models import Sequential
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dense
from keras.optimizers import Adam
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from PIL import Image
from imutils import paths
import numpy as np
import argparse
import os

# grab all image paths in the input dataset directory, then initialize
# our list of images and corresponding class labels
print("[INFO] loading images...")
imagePaths = paths.list_images('/content/scenes/')
nstPaths = paths.list_images('/content/nst/')
data = []
labels = []
nstdata = []
nstlabels = []
# loop over our input images
for imagePath in imagePaths:
	# load the input image from disk, resize it to 32x32 pixels, scale
	# the pixel intensities to the range [0, 1], and then update our
	# images list
	image = Image.open(imagePath)
	image = np.array(image.resize((32, 32))) / 255.0
	data.append(image)

	# extract the class label from the file path and update the
	# labels list
	label = imagePath.split(os.path.sep)[-2]
	labels.append(label)

for nstPath in nstPaths:
	# load the input image from disk, resize it to 32x32 pixels, scale
	# the pixel intensities to the range [0, 1], and then update our
	# images list
	image1 = Image.open(nstPath)
	image1 = np.array(image1.resize((32, 32))) / 255.0
	nstdata.append(image1)

	# extract the class label from the file path and update the
	# labels list
	label1 = nstPath.split(os.path.sep)[-1]
	nstlabels.append(label1) 
  
# encode the labels, converting them from strings to integers
lb = LabelBinarizer()
lb1 =  LabelBinarizer()
labels = lb.fit_transform(labels)
nstlabels = lb1.fit_transform(nstlabels)
# perform a training and testing split, using 75% of the data for
# training and 25% for evaluation
(trainX, testX, trainY, testY) = train_test_split(np.array(data),
	np.array(labels), test_size=0.25)



# define our Convolutional Neural Network architecture
model = Sequential()
model.add(Conv2D(8, (3, 3), padding="same", input_shape=(32, 32, 3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Conv2D(16, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Conv2D(32, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Flatten())
model.add(Dense(3))
model.add(Activation("softmax"))

# train the model using the Adam optimizer
print("[INFO] training network...")
opt = Adam(lr=1e-3, decay=1e-3 / 50)
model.compile(loss="categorical_crossentropy", optimizer=opt,
	metrics=["accuracy"])
H = model.fit(trainX, trainY, validation_data=(testX, testY),
	epochs=50, batch_size=32)
nstdata
# evaluate the network
print("[INFO] evaluating network...")
predictions = model.predict(testX, batch_size=32)
print(model.predict_classes(np.array(nstdata)[:,:, :, :3]))
print(classification_report(testY.argmax(axis=1),predictions.argmax(axis=1), target_names=lb.classes_))

Using TensorFlow backend.


[INFO] loading images...


W0621 11:10:32.601557 140369926252416 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0621 11:10:32.645570 140369926252416 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0621 11:10:32.654544 140369926252416 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0621 11:10:32.706049 140369926252416 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.

W0621 11:10:32.784773 140369926252416 deprecation_wrapp

[INFO] training network...


W0621 11:10:33.024776 140369926252416 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.



Train on 646 samples, validate on 216 samples
Epoch 1/50
646/646 [==============================] - 2s 2ms/step - loss: 1.0365 - acc: 0.4226 - val_loss: 0.9372 - val_acc: 0.6528
Epoch 2/50
646/646 [==============================] - 1s 917us/step - loss: 0.7523 - acc: 0.6858 - val_loss: 0.7139 - val_acc: 0.6944
Epoch 3/50
646/646 [==============================] - 1s 881us/step - loss: 0.5879 - acc: 0.7539 - val_loss: 0.5321 - val_acc: 0.8102
Epoch 4/50
646/646 [==============================] - 1s 903us/step - loss: 0.4654 - acc: 0.8344 - val_loss: 0.4845 - val_acc: 0.7963
Epoch 5/50
646/646 [==============================] - 1s 884us/step - loss: 0.4141 - acc: 0.8545 - val_loss: 0.4484 - val_acc: 0.8102
Epoch 6/50
646/646 [==============================] - 1s 928us/step - loss: 0.3898 - acc: 0.8514 - val_loss: 0.4344 - val_acc: 0.8241
Epoch 7/50
646/646 [==============================] - 1s 908us/step - loss: 0.3884 - acc: 0.8529 - val_loss: 0.4001 - val_acc: 0.8611
Epoch 8/50
646/646